<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/LinearRegression_MultiVariable_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Python

func getArrayDataset() -> ([[Float]], [Float]) {
    let np = Python.import("numpy")
    let pd = Python.import("pandas")
    let io = Python.import("io")
    let requests = Python.import("requests")

    let url="https://raw.githubusercontent.com/Dataweekends/zero_to_deep_learning_video/master/data/weight-height.csv"
    let s = requests.get(url).content
    let df = pd.read_csv(io.StringIO(s.decode("utf-8")))

    let dummies = pd.get_dummies(df[["Gender"]])
    let trasnformed = pd.concat([df[["Height", "Weight"]], dummies], 1)
    // print(trasnformed)

    let X1 = trasnformed[["Height"]].values
    let X2 = trasnformed[["Gender_Female"]].values
    let X3 = trasnformed[["Gender_Male"]].values
    let Y = trasnformed[["Weight"]].values
  
    let x1Array = X1.tolist().flatMap{ $0.map{ Float($0)! }}
    let x2Array = X2.tolist().flatMap{ $0.map{ Float($0)! }}
    let x3Array = X3.tolist().flatMap{ $0.map{ Float($0)! }}
    let yArray = Y.tolist().flatMap{ $0.map{ Float($0)! }}
  
    return ([x1Array, x2Array, x3Array], yArray)
}

In [0]:
func minMaxNormalized<T: BinaryFloatingPoint>(_ vector : [T]) -> [T] {
    guard let (min, max) = minimumMaximum(vector) else { return vector }
    return vector.map{ ($0 - min) / (max - min) }
}

func minimumMaximum<T: Comparable>(_ vector: [T]) -> (minimum: T, maximum: T)? {
    guard var minimum = vector.first else { return nil }
    var maximum = minimum

    let start = vector.count % 2
    for i in stride(from: start, to: vector.count, by: 2) {
        let (first, second) = (vector[i], vector[i+1])

        if first > second {
            if first > maximum {
                maximum = first
            }
            if second < minimum {
                minimum = second
            }
        } else {
            if second > maximum {
                maximum = second
            }
            if first < minimum {
                minimum = first
            }
        }
    }

    return (minimum, maximum)
}

In [0]:
let (xArray, yVector) = getArrayDataset()

let (minHeight, maxHeight) = minimumMaximum(xArray[0])!
let (minWeight, maxWeight) = minimumMaximum(yVector)!

let yNormalizedVector = minMaxNormalized(yVector)
let xNormalizedArray = [minMaxNormalized(xArray[0]), xArray[1], xArray[2]]

let count = xNormalizedArray[0].count * 3
var xNormalizedVector = [Float](repeating: 0, count: count)

for i in stride(from: 0, to: count, by: 3) {
    xNormalizedVector[i] = xNormalizedArray[0][i/3]
    xNormalizedVector[i+1] = xNormalizedArray[1][i/3]
    xNormalizedVector[i+2] = xNormalizedArray[2][i/3]
}

In [0]:
import TensorFlow

In [0]:
@differentiable()
func linear(input: Tensor<Float>) -> Tensor<Float> {
    return input
}

struct LinearRegression: Layer {
    var l1: Dense<Float>
    init(variables: Int = 1) {
        l1 = Dense<Float>(inputSize: variables, outputSize: 1, activation: linear)
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>) -> Tensor<Float> {
        return l1.applied(to: input)
    }
}

In [6]:
let x = Tensor<Float>(shape: [Int32(10000), 3], scalars: xNormalizedVector)
let y = Tensor<Float>(shape: [Int32(10000), 1], scalars: yNormalizedVector)

print(x.shape)
print(y.shape)

TensorShape(dimensions: [10000, 3])
TensorShape(dimensions: [10000, 1])


In [7]:
let optimizer = SGD<LinearRegression, Float>(learningRate: 0.03)
var model = LinearRegression(variables: 3)


for epoch in 1...2000 {
    let (cost, 𝛁model) = model.valueWithGradient { m -> Tensor<Float> in
        let ŷ = m.applied(to: x)
        return meanSquaredError(predicted: ŷ, expected: y)
    }
    optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
  
    if epoch % 100 == 0 {
        print("Epoch: \(epoch) Cost: \(cost)")
    }
}

Epoch: 100 Cost: 0.011159992
Epoch: 200 Cost: 0.010100178
Epoch: 300 Cost: 0.009171858
Epoch: 400 Cost: 0.008355156
Epoch: 500 Cost: 0.0076366393
Epoch: 600 Cost: 0.0070045106
Epoch: 700 Cost: 0.0064483797
Epoch: 800 Cost: 0.005959111
Epoch: 900 Cost: 0.0055286638
Epoch: 1000 Cost: 0.005149969
Epoch: 1100 Cost: 0.004816803
Epoch: 1200 Cost: 0.004523691
Epoch: 1300 Cost: 0.0042658197
Epoch: 1400 Cost: 0.0040389504
Epoch: 1500 Cost: 0.003839357
Epoch: 1600 Cost: 0.0036637615
Epoch: 1700 Cost: 0.0035092751
Epoch: 1800 Cost: 0.0033733642
Epoch: 1900 Cost: 0.0032537922
Epoch: 2000 Cost: 0.0031485963


In [8]:
let height = Float(76.8)
let heightTensor = TensorElementLiteral<Float>(floatLiteral: (height - minHeight) / (maxHeight - minHeight))

let normPredictedWeight = model.applied(to: [[heightTensor, 0, 1]])[0][0]  //Height, Female, Male
let predictedWeight = (normPredictedWeight * (maxWeight - minWeight)) + minWeight

print(predictedWeight)

217.8783
